In [1]:
#处理得分率
import pandas as pd
import chardet
# 读取数据
df_a = pd.read_excel('b1_t_stat_exam_answer.xlsx')
df_b = pd.read_excel('b1_t_stat_exam_relation.xlsx')


In [2]:
# 获取发布分数
df_b = df_b[['exam_id', 'score']]  # 假设关系表中有 exam_id 和 score 列
df_a = pd.merge(df_a, df_b, on='exam_id', how='left')

# 计算新分数
df_a['new_score'] = df_a['score_x'] / df_a['score_y'] * 100

# 根据 personid、clazzid 和 exam_id 替换原始分数
df_a['score'] = df_a['new_score'].where(df_a['new_score'].notna(), df_a['score_x'])

# 选择需要的列
df_a = df_a[['personid', 'clazzid', 'exam_id', 'score']]

# 计算班级平均分和标准差
class_stats = df_a.groupby('clazzid')['score'].agg(['mean', 'std']).reset_index()
class_stats.columns = ['clazzid', 'class_mean', 'class_std']

# 将班级统计信息与原始数据合并
df_a = pd.merge(df_a, class_stats, on='clazzid', how='left')

# 计算班级排名
df_a['class_rank'] = df_a.groupby('clazzid')['score'].rank(ascending=False, method='min')

# 显示处理后的数据
print(df_a)

       personid   clazzid  exam_id       score  class_mean  class_std  \
0      28405787   9665961  1109815   85.000000   65.704545  22.821430   
1      28405787   9665961  1142618   52.000000   65.704545  22.821430   
2      28405855   8009388   170806  100.000000   81.401051  18.754142   
3      28405855   8009388   184187  100.000000   81.401051  18.754142   
4      28405855   8009388   196723  100.000000   81.401051  18.754142   
...         ...       ...      ...         ...         ...        ...   
75126  40301258  11197704  1125013    0.000000   38.422824  22.030347   
75127  40301259  11197703  1125413   73.134328   53.117542  24.843707   
75128  40301259  11197703  1132261   73.134328   53.117542  24.843707   
75129  40301259  11197703  1139269   55.721393   53.117542  24.843707   
75130  40301259  11197703  1146789   62.500000   53.117542  24.843707   

       class_rank  
0            10.0  
1            33.0  
2             1.0  
3             1.0  
4             1.0  
...

In [3]:
#处理个人考试方差/平均值/分数最大值/最小值
person_class_stats = df_a.groupby(['personid', 'clazzid']).agg(
    mean_score=('score', 'mean'),
    std_score=('score', 'std'),
    max_score=('score', 'max'),
    min_score=('score', 'min'),
).reset_index()

person_class_stats.fillna(0, inplace=True)
#NaN没有方差，只参加一次考试
print(person_class_stats)

       personid   clazzid  mean_score  std_score   max_score  min_score
0      28405787   9665961   68.500000  23.334524   85.000000  52.000000
1      28405855   8009388   95.000000  10.000000  100.000000  80.000000
2      28405934   9665960   95.000000   7.071068  100.000000  90.000000
3      28405970   9665961   82.500000  24.748737  100.000000  65.000000
4      28406016   6137475   73.532609  21.728621  100.000000  40.000000
...         ...       ...         ...        ...         ...        ...
14387  40047834  11089034   88.308458   2.462561   90.049751  86.567164
14388  40301256  11197674   12.000000  11.313708   20.000000   4.000000
14389  40301258  11197703   19.912935  11.731197   32.835821   9.950249
14390  40301258  11197704    0.000000   0.000000    0.000000   0.000000
14391  40301259  11197703   66.122512   8.556421   73.134328  55.721393

[14392 rows x 6 columns]


In [4]:
# 统计同一个 personid 和 clazzid 出现的 examid 次数
person_clazz_exam_count = df_a.groupby(['personid', 'clazzid']).size().reset_index(name='person_clazz_exam_count')

# 统计同一个 clazzid 出现的不同 examid 次数
clazz_unique_exam_count = df_a.groupby(['clazzid'])['exam_id'].nunique().reset_index(name='clazz_unique_exam_count')

# 合并数据
merged_result = pd.merge(person_clazz_exam_count, clazz_unique_exam_count, on='clazzid', how='left')

# 计算 count_percentage
merged_result['count_percentage'] = (merged_result['person_clazz_exam_count'] / merged_result['clazz_unique_exam_count']).fillna(0)

# 打印结果
print(merged_result)

       personid   clazzid  person_clazz_exam_count  clazz_unique_exam_count  \
0      28405787   9665961                        2                        2   
1      28405855   8009388                        4                        4   
2      28405934   9665960                        2                        2   
3      28405970   9665961                        2                        2   
4      28406016   6137475                       16                       16   
...         ...       ...                      ...                      ...   
14387  40047834  11089034                        2                        2   
14388  40301256  11197674                        2                        3   
14389  40301258  11197703                        4                        4   
14390  40301258  11197704                        1                        4   
14391  40301259  11197703                        4                        4   

       count_percentage  
0              1.000000  

In [5]:
# 合并数据
merged_score_result = pd.merge(merged_result, person_class_stats, on=['clazzid','personid'], how='left')
merged_score_result

,personid,clazzid,person_clazz_exam_count,clazz_unique_exam_count,count_percentage,mean_score,std_score,max_score,min_score
0,28405787,9665961,2,2,1.000000,68.500000,23.334524,85.000000,52.000000
1,28405855,8009388,4,4,1.000000,95.000000,10.000000,100.000000,80.000000
2,28405934,9665960,2,2,1.000000,95.000000,7.071068,100.000000,90.000000
3,28405970,9665961,2,2,1.000000,82.500000,24.748737,100.000000,65.000000
4,28406016,6137475,16,16,1.000000,73.532609,21.728621,100.000000,40.000000
...,...,...,...,...,...,...,...,...,...
14387,40047834,11089034,2,2,1.000000,88.308458,2.462561,90.049751,86.567164
14388,40301256,11197674,2,3,0.666667,12.000000,11.313708,20.000000,4.000000
14389,40301258,11197703,4,4,1.000000,19.912935,11.731197,32.835821,9.950249
14390,40301258,11197704,1,4,0.250000,0.000000,0.000000,0.000000,0.000000


In [6]:
#处理活动率（每个同学在每个班的活动率）
df_a=pd.read_excel('b1_t_stat_activity_log.xlsx')
df_b=pd.read_excel('b1_t_stat_widget_log.xlsx')

activity_counts_a = df_a.groupby(['personid', 'clazzid'])['activity_id'].count().reset_index()
activity_counts_a.columns = ['personid', 'clazzid', 'activity_count_a']


activity_counts_b = df_b.groupby([ 'clazzid'])['activity_id'].count().reset_index()
activity_counts_b.columns = [ 'clazzid', 'activity_count_b']


In [7]:

merged_activity_result = pd.merge(activity_counts_a, activity_counts_b, on=['clazzid'], how='left')
merged_activity_result.fillna(0, inplace=True)

print(merged_activity_result)

       personid  clazzid  activity_count_a  activity_count_b
0      28405745  5964880                 1                 4
1      28405745  6838708                 2                 5
2      28405756  6136781                 2                 7
3      28405756  6137100                 1                 2
4      28405756  6137477                 1                 1
...         ...      ...               ...               ...
95914  42880549  9665471                 1                 4
95915  42880555  9665471                 1                 4
95916  42880660  9665471                 1                 4
95917  42945396  9663524                 1                 3
95918  43006519  9663730                 1                23

[95919 rows x 4 columns]


In [8]:
#合并数据
# 计算 count_activity_percentage
merged_activity_result['count_activity_percentage'] = (merged_activity_result['activity_count_a'] /merged_activity_result['activity_count_b']).fillna(0)

merged_result = pd.merge(merged_activity_result, merged_score_result, on=['clazzid','personid'], how='left')
merged_result

,personid,clazzid,activity_count_a,activity_count_b,count_activity_percentage,person_clazz_exam_count,clazz_unique_exam_count,count_percentage,mean_score,std_score,max_score,min_score
0,28405745,5964880,1,4,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28405745,6838708,2,5,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28405756,6136781,2,7,0.285714,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28405756,6137100,1,2,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28405756,6137477,1,1,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95914,42880549,9665471,1,4,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95915,42880555,9665471,1,4,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95916,42880660,9665471,1,4,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95917,42945396,9663524,1,3,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#追加选课名字以及courseid
# 从 Excel 文件读取数据
df_course_person = pd.read_excel('b1_t_stat_course_person.xlsx')
df_clazz = pd.read_excel('b1_t_stat_clazz.xlsx')
df_course = pd.read_excel('b1_t_stat_course.xlsx')

In [17]:
# 合并数据
merged_df = pd.merge(df_course_person[['personid','clazzid','role']], df_clazz[['clazzid', 'courseid', 'semester','student_count']], on='clazzid')

# 显示合并后的数据
print(merged_df)

         personid   clazzid  role  courseid  semester  student_count
0        28406646  10677390     1   2030061    202201             51
1        28411207  10677390     3   2030061    202201             51
2        28411234  10677390     3   2030061    202201             51
3        28411255  10677390     3   2030061    202201             51
4        28411256  10677390     3   2030061    202201             51
...           ...       ...   ...       ...       ...            ...
1048570  32912466   9664881     3   1764566    202201             27
1048571  32912484   9664881     3   1764566    202201             27
1048572  32912485   9664881     3   1764566    202201             27
1048573  32912486   9664881     3   1764566    202201             27
1048574  32912487   9664881     3   1764566    202201             27

[1048575 rows x 6 columns]


In [18]:
merged_df = pd.merge(merged_df, df_course[['name','courseid','type']], on='courseid')

# 打印合并后的结果
print(merged_df)


         personid   clazzid  role  courseid  semester  student_count    name  \
0        28406646  10677390     1   2030061    202201             51    拓展运动   
1        28411207  10677390     3   2030061    202201             51    拓展运动   
2        28411234  10677390     3   2030061    202201             51    拓展运动   
3        28411255  10677390     3   2030061    202201             51    拓展运动   
4        28411256  10677390     3   2030061    202201             51    拓展运动   
...           ...       ...   ...       ...       ...            ...     ...   
1048570  40047480   9664730     3   2460133    202201             80  海洋知识概论   
1048571  28409823   9664801     1   2460134    202201              3   应用藻类学   
1048572  28425380   9664801     3   2460134    202201              3   应用藻类学   
1048573  28425387   9664801     3   2460134    202201              3   应用藻类学   
1048574  28425397   9664801     3   2460134    202201              3   应用藻类学   

         type  
0           2  
1      

In [19]:
# 合并数据
merged_result = pd.merge(merged_df,merged_result,on=['clazzid','personid'], how='left')
# 打印结果
print(merged_result)

         personid   clazzid  role_x  courseid_x  semester_x  student_count_x  \
0        28406646  10677390       1     2030061      202201               51   
1        28411207  10677390       3     2030061      202201               51   
2        28411234  10677390       3     2030061      202201               51   
3        28411255  10677390       3     2030061      202201               51   
4        28411256  10677390       3     2030061      202201               51   
...           ...       ...     ...         ...         ...              ...   
1048570  40047480   9664730       3     2460133      202201               80   
1048571  28409823   9664801       1     2460134      202201                3   
1048572  28425380   9664801       3     2460134      202201                3   
1048573  28425387   9664801       3     2460134      202201                3   
1048574  28425397   9664801       3     2460134      202201                3   

         name_x  type_x  role_y  course

In [20]:
# 将结果导出到 CSV 文件
merged_result.to_csv('final_result.csv', index=False)